<a href="https://colab.research.google.com/github/akhii-leesh/Next-word-predictor/blob/main/NextWordPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hmmlearn nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

# Download NLTK tokenizer and stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv("/content/Shakespeare_data.csv")
df['PlayerLine'] = df['PlayerLine'].str.replace('[^\w\s]', '')
df.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [ ]:
# Combine all the text into a single corpus
corpus = ' '.join(df['PlayerLine'].tolist())

# Tokenize the corpus into words
tokens = nltk.word_tokenize(corpus)

# Remove punctuation tokens
words = [word.lower() for word in tokens if word.isalpha()]

# Create a vocabulary of unique words
vocab = list(set(words))
vocab_size = len(vocab)

In [ ]:
# Map words to integer IDs
word_to_id = {word: i for i, word in enumerate(vocab)}
id_to_word = {i: word for i, word in enumerate(vocab)}

# Prepare sequences of word IDs for training
sequences = [word_to_id[word] for word in words]

# Split the data into training and test sets
train_sequences, test_sequences = train_test_split(sequences, test_size=0.2, random_state=42)

In [ ]:
from hmmlearn import hmm
import numpy as np

# Number of hidden states (e.g., parts of speech or sentence structures)
n_hidden_states = 10

# Prepare the sequences as input for the HMM
X_train = np.array(train_sequences).reshape(-1, 1)  # Reshape for HMM input

# Define and train the Hidden Markov Model
model = hmm.MultinomialHMM(n_components=n_hidden_states, n_iter=100)
model.fit(X_train)

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


MultinomialHMM(n_components=10, n_iter=100,
               n_trials=array([ 3121,  7107,  1180, ..., 13555, 18227,  3464]),
               random_state=RandomState(MT19937) at 0x7E8908502040)

In [ ]:
def predict_next_word(current_word, model, id_to_word, word_to_id):
    current_word_id = word_to_id.get(current_word, None)

    if current_word_id is None:
        return "Word not in vocabulary"

    # Predict the next word based on the current word
    next_state = model.predict(np.array([[current_word_id]]))
    next_word_id = np.argmax(model.transmat_[next_state])

    return id_to_word[next_word_id]

In [ ]:
# Example usage:
current_word = "love"
next_word = predict_next_word(current_word, model, id_to_word, word_to_id)
print(f"Next word prediction for '{current_word}': {next_word}")

Next word prediction for 'love': fright
